# Get similar artists via last.fm
* Documentation top: https://www.last.fm/api/intro
* artist.getSimilar: https://www.last.fm/api/show/artist.getSimilar

In [1]:
import requests

with open('lastfm_api_key.txt') as f:
    lastfm_api_key = f.read()

In [2]:
def ask_lastfm_similar_artists(artist):
    url = 'http://ws.audioscrobbler.com/2.0/'

    data = {'method': 'artist.getsimilar', 
            'artist': artist, 
            'autocorrect': 0,
            'api_key': lastfm_api_key, 
            'format': 'json'}
    
    response = requests.post(url, data=data)
    
    if response.status_code == 200:
        r = response.json()
        similar_list = r['similarartists']['artist']
        return similar_list
    
    print('Sorry, some kind of error with status code {}'.format(response.status_code))
    return None

In [3]:
def print_lastfm_response(similar_list):
    print('# of similar artists: {:,}'.format(len(similar_list)))
    for sim in similar_list:
        print(sim['match'], sim['name'])  # match is b/w 0 and 1 (float)

In [4]:
def parse_lastfm_response(similar_list):
    list_ = []
    for sim in similar_list:
        list_.append((sim['name'], float(sim['match'])))  # match is b/w 0 and 1 (float)
    return list_

In [5]:
def get_top_n_similar_artists(artist, n=5, match_min=0, include_src=False):
    similar_list = ask_lastfm_similar_artists(artist)    
    if similar_list is None:
        return

    list_ = parse_lastfm_response(similar_list)
    output = []
    for i, name_match in enumerate(list_):
        name, match = name_match
        if (i < n) and (match >= match_min):
            if include_src:
                output.append((artist, name))
            else:
                output.append(name)
    return output

In [6]:
get_top_n_similar_artists('jimi hendrix', n=10, match_min=0.5)

['The Jimi Hendrix Experience', 'Cream', 'Led Zeppelin', 'The Doors']

In [6]:
def get_top_n_linked_artists(artist, n_min=50, match_min=0.5, n_per_artist=5, include_src=False):
    if include_src:
        output = [(artist, artist)]
    else:
        output = [artist]
        
    queue = [artist]
    while True:
        if len(queue) == 0:
            print('Couldn\'t find {:,} artists...'.format(n_min))
            break
        
        focus = queue[0]
        list_ = get_top_n_similar_artists(focus, n=n_per_artist, match_min=match_min, include_src=False)
        
        to_add = []
        for elem in list_:  # Check if it's already checked/added
            if include_src:
                if elem.lower() in [o.lower() for _, o in output]:
                    continue
                to_add.append((focus, elem))
            else:
                if elem.lower() in [o.lower() for o in output]:
                    continue
                to_add.append(elem)
        
        output.extend(to_add)
        if len(output) >= n_min:
            break
        
        if include_src:
            queue.extend([elem for _, elem in to_add])
        else:
            queue.extend(to_add)       
        queue = queue[1:]  # Remove current artist
            
    return output

In [9]:
get_top_n_linked_artists('david bowie', n_min=300, match_min=0, n_per_artist=10, include_src=True)

[('david bowie', 'david bowie'),
 ('david bowie', 'Tin Machine'),
 ('david bowie', 'Lou Reed'),
 ('david bowie', 'Iggy Pop'),
 ('david bowie', 'Roxy Music'),
 ('david bowie', 'The Velvet Underground'),
 ('david bowie', 'T. Rex'),
 ('david bowie', 'Queen & David Bowie'),
 ('david bowie', 'The Rolling Stones'),
 ('david bowie', 'Queen'),
 ('david bowie', 'Brian Eno'),
 ('Tin Machine', 'The Cult'),
 ('Tin Machine', 'Porno for Pyros'),
 ('Tin Machine', 'Cheap Trick'),
 ('Tin Machine', "Jane's Addiction"),
 ('Tin Machine', 'Nick Cave & The Bad Seeds'),
 ('Tin Machine', 'R.E.M.'),
 ('Tin Machine', 'Therapy?'),
 ('Tin Machine', 'Robert Plant'),
 ('Tin Machine', 'Morrissey'),
 ('Lou Reed', 'Lou Reed & John Cale'),
 ('Lou Reed', 'John Cale'),
 ('Lou Reed', 'Neil Young'),
 ('Lou Reed', 'Patti Smith'),
 ('Lou Reed', 'The Stooges'),
 ('Lou Reed', 'Bob Dylan'),
 ('Iggy Pop', 'Iggy Pop & James Williamson'),
 ('Iggy Pop', 'The Clash'),
 ('Iggy Pop', 'Ramones'),
 ('Iggy Pop', 'Sex Pistols'),
 ('Iggy P